In [15]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

In [16]:
# Enter current gameweek 
gameweek = 14

## Collect data

In [17]:
# Initialize an empty list to store the data from each gameweek
all_gameweeks = []

# Loop through each gameweek
for i in range(1, gameweek + 1):  # Adjusting the range to start from 1 to gameweek
    # Read the CSV for the current gameweek
    gameweek_data = pd.read_csv(rf'C:\Users\thoma\Code\Projects\Fantasy-Premier-League\Data\Players\Seperate_GW\GW_{i}.csv')
    
    # Append the current gameweek data to the list
    all_gameweeks.append(gameweek_data)

# Concatenate all dataframes in the list into a single dataframe
data = pd.concat(all_gameweeks, axis=0, ignore_index=True)

# Drop unnamed column
data = data.drop(columns = ['Unnamed: 0'])

## Calculate form

In [18]:
# Sort dataset by Player ID and Gameweek
final_data = data.sort_values(by=['Player ID', 'Gameweek'])

# Define the rolling window size
number_of_games = 4

# Calculate the rolling average of GW_Points over the specified number_of_games
final_data["Form"] = (
    final_data
    .groupby("Player ID")["GW Points"]
    .transform(lambda x: x.rolling(window=number_of_games).mean().round(3))
)

# Choose important columns
columns = [
    'Player ID', 'Name', 'Last_Name', 'Team', 'Position', 'Cost_Today',
    'GW Points', 'Form', 'Gameweek'
]

final_data = final_data[columns]

## Add upcoming fixtures 

In [19]:
# Add fixture list into spreadsheet
fixtures = pd.read_csv(r'C:\Users\thoma\Code\Projects\Fantasy-Premier-League\Data\Fixtures\Schedule\Fixtures.csv')

# Merge on fixture list
final_data = final_data.merge(fixtures, on= 'Team')

# Drop unneeded gameweek columns
def drop_gw_columns(final_data, gameweek):
    # Create lists of columns to drop
    columns_to_drop = [f'GW{i}' for i in range(1, gameweek + 1)] + [f'GW{i}' for i in range(gameweek + 6, 39)]
    
    # Drop columns if they exist in the DataFrame
    final_data = final_data.drop(columns=[col for col in columns_to_drop if col in final_data.columns], errors='ignore')
    return final_data

# Run the loop
data = drop_gw_columns(final_data, gameweek)

# Current gameweek

In [20]:
# filter on current gameweek
today = data['Gameweek'].isin([gameweek])
data = data[today]

In [21]:
data.iloc[500:570]

,Player ID,Name,Last_Name,Team,Position,Cost_Today,GW Points,Form,Gameweek,GW15,GW16,GW17,GW18,GW19
7012,501,Oliver,Skipp,Leicester,MID,48,0,0.50,14,BHA (H),NEW (A),WOL (H),LIV (A),MCI (H)
7026,502,Manor,Solomon,Spurs,MID,54,0,0.00,14,CHE (H),SOU (A),LIV (H),NFO (A),WOL (H)
7040,503,Son,Son,Spurs,MID,99,1,2.75,14,CHE (H),SOU (A),LIV (H),NFO (A),WOL (H)
7054,504,Djed,Spence,Spurs,DEF,44,0,0.25,14,CHE (H),SOU (A),LIV (H),NFO (A),WOL (H)
7068,505,Destiny,Udogie,Spurs,DEF,49,2,2.75,14,CHE (H),SOU (A),LIV (H),NFO (A),WOL (H)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7922,566,Jørgen,Strand Larsen,Wolves,FWD,56,2,5.00,14,WHU (A),IPS (H),LEI (A),MUN (H),TOT (A)
7936,567,Toti António,Toti,Wolves,DEF,43,1,3.00,14,WHU (A),IPS (H),LEI (A),MUN (H),TOT (A)
7950,568,Arijanet,Muric,Ipswich,GK,44,2,2.75,14,BOU (H),WOL (A),NEW (H),ARS (A),CHE (H)
7964,569,Jaden,Jaden,Aston Villa,MID,53,1,1.00,14,SOU (H),NFO (A),MCI (H),NEW (A),BHA (H)


## Fixture Difficulty

In [22]:
# Import improve fixture difficulty 
difficulty = pd.read_csv(r'C:\Users\thoma\Code\Projects\Fantasy-Premier-League\Data\Fixtures\Difficulty_ratings\Model\FD_combined\Current_FD.csv', index_col=0)

# Create a mapping dictionary from fixture difficulty
mapping = difficulty.set_index(['Opponent', 'Position'])['FD_combined'].to_dict()

# Map difficulty for NGWs (next gameweeks) using Team and Position
for i in range(1, 6):  # NGW1 to NGW5
    data[f'NGW{i}'] = data.apply(lambda row: mapping.get((row.iloc[8 + i], row.iloc[4]), None), axis=1)

# Loop to create FDI_1 to FDI_5, summing up the values from F_1 to F_i
for i in range(1, 6):
    # Create FDI_i by summing the appropriate columns
    data[f'F_{i}'] = data[[f'NGW{j}' for j in range(1, i+1)]].sum(axis=1)

# Calculate accumulated FD_index for up to next 5 gameweeks
for i in range(1, 6):
    data[f'FDI_{i}'] = round(data.iloc[:, 7] / data.iloc[:, 18 + i], 4)

## Optimization

In [23]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Model_1

# This model uses the FD_index to choose the best players. The FD_index is a 
# simple calculation of current form/upcoming fixture difficulty. The fixture
# difficulty can be planned for up to 5 weeks. The model will choose the players
# that have the highest form per lowest fixture difficulty, and optimally select
# a team given the constraints of budget, position and team limit. 

# Define constants
BUDGET = 800 # Choose your budget (1000 = £100m)
WEEKS = 3 # Choose how many weeks you want to prepare for between 1 and 5
GK = 1 # Goalkeepers required (Choose between 0 and 2)
DEF = 3 # Defenders required (Choose between 0 and 5)
MID = 4 # Midfielders required (Choose between 0 and 5)
FWD = 3
 #  Forwards required (Choose between 0 and 3)

# Use dataset
data = data

In [24]:
# Dynamically create the column name based on the number of weeks
column_name = f'FDI_{WEEKS}'

# Filter out players with FD_index == 0 to avoid selecting them
data = data[data[column_name] > 0]

# Create lists of key variables
names = data.Last_Name.tolist()
teams = data.Team.tolist()
positions = data.Position.tolist()
prices = data.Cost_Today.tolist()
FD_index = data[column_name].tolist()

# Initialize the problem
prob = LpProblem("FPL_Player_Choices", LpMaximize)

# Create binary variables for players
players = [LpVariable(f"player_{i}", cat="Binary") for i in range(len(data))]

# Define the objective function: maximize the sum of FD_index for selected players
prob += lpSum(players[i] * FD_index[i] for i in range(len(data)))

# Budget constraint: the sum of selected players' prices must be <= BUDGET
prob += lpSum(players[i] * prices[i] for i in range(len(data))) <= BUDGET

# Budget constraint: the sum of selected players' prices must be <= BUDGET
prob += lpSum(players[i] * prices[i] for i in range(len(data))) >= (BUDGET - 75)

# Position constraints: enforce exact limits for each position
prob += lpSum(players[i] for i in range(len(data)) if positions[i] == 'GK') == GK
prob += lpSum(players[i] for i in range(len(data)) if positions[i] == 'DEF') == DEF 
prob += lpSum(players[i] for i in range(len(data)) if positions[i] == 'MID') == MID 
prob += lpSum(players[i] for i in range(len(data)) if positions[i] == 'FWD') == FWD  

# Club constraint: each team can have at most 3 players
for club in data.Team.unique():
    prob += lpSum(players[i] for i in range(len(data)) if teams[i] == club) <= 3

# Solve the problem
prob.solve()

# Create a list of selected players
selected_players = []
for v in prob.variables():
    if v.varValue != 0:
        index = int(v.name.split("_")[1])
        player_info = {
            'Name': names[index],
            'Team': teams[index],
            'Position': positions[index],
            'FD_Index': FD_index[index],
            'Price': prices[index],
        }
        selected_players.append(player_info)

# Convert selected players to a DataFrame for a better display
selected_players_df = pd.DataFrame(selected_players)

# Display the DataFrame
print(selected_players_df)

# Display the total cost and index
print(f'Total Team Cost:', sum(selected_players_df.Price))
print(f'Total Team Index', sum(selected_players_df.FD_Index))

           Name         Team Position  FD_Index  Price
0          Saka      Arsenal      MID    0.9000    104
1        Saliba      Arsenal      DEF    0.8929     60
2      Kelleher    Liverpool       GK    0.5500     45
3       M.Salah    Liverpool      MID    1.0357    132
4   Wan-Bissaka     West Ham      DEF    0.8750     45
5          Isak    Newcastle      FWD    1.0417     85
6      J.Timber      Arsenal      DEF    0.8214     55
7         Cunha       Wolves      FWD    1.3333     71
8      Kluivert  Bournemouth      MID    1.0000     54
9    João Pedro     Brighton      FWD    1.2500     58
10       Mitoma     Brighton      MID    0.9583     65
Total Team Cost: 774
Total Team Index 10.6583
